In [1]:
# Import necessary libraries
import datetime
import time
import yfinance as yf
import requests
import pandas as pd
import numpy as np
import tweepy
from scipy.signal import find_peaks
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt



# Define the stock symbol (Ensure only Intel - INTC is used)
stock = "INTC"

# Fetch stock data with error handling
try:
    print(f"⏳ Fetching stock data for {stock}...")

    # Attempt to download stock data (first try 6 months, fallback to 3 months)
    data = yf.download(stock, period="6mo", interval="1d", progress=False)

    # If no data, try fetching only 3 months
    if data.empty:
        print(f"⚠️ No data for 6 months. Trying 3 months instead for {stock}...")
        data = yf.download(stock, period="3mo", interval="1d", progress=False)

    # Ensure data is valid
    if data.empty:
        raise ValueError(f"⚠️ Error: No valid stock data retrieved for {stock}. Try a shorter time period or check API status.")

    # Extract OHLC and Volume
    ohlc_data = data[['Open', 'High', 'Low', 'Close', 'Volume']]

    # Drop rows with missing values
    ohlc_data.dropna(inplace=True)

    # Save Intel's data to CSV
    ohlc_data.to_csv(f"{stock}_Stock_Data.csv")
    print(f"📂 {stock} stock data saved to '{stock}_Stock_Data.csv'.")

    # Display first few rows
    print(f"✅ {stock} stock data successfully retrieved:")
    print(ohlc_data.tail(10))  # Show first 10 rows

except Exception as e:
    print(f"⚠️ Error fetching stock data for {stock}: {e}")

⏳ Fetching stock data for INTC...
YF.download() has changed argument auto_adjust default to True
📂 INTC stock data saved to 'INTC_Stock_Data.csv'.
✅ INTC stock data successfully retrieved:
Price            Open       High        Low      Close     Volume
Ticker           INTC       INTC       INTC       INTC       INTC
Date                                                             
2025-03-17  24.420000  26.150000  24.370001  25.690001  160723600
2025-03-18  25.549999  26.410000  25.389999  25.920000  132488800
2025-03-19  24.900000  25.230000  24.030001  24.120001  135491400
2025-03-20  23.770000  24.309999  23.730000  23.959999   68937600
2025-03-21  23.600000  24.700001  23.490000  24.260000   94110700
2025-03-24  24.719999  24.990000  24.129999  24.219999   69169000
2025-03-25  24.180000  24.750000  24.110001  24.200001   51743300
2025-03-26  24.250000  24.420000  23.250000  23.420000   59496700
2025-03-27  23.290001  23.760000  22.860001  23.620001   57720600
2025-03-28  23.3500

In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download VADER Lexicon
import nltk
nltk.download('vader_lexicon')

# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer() 


# Testing VADER samples for scores
# Test VADER with various texts
texts = [
    "This is amazing!",
    "This is the worst experience ever.",
    "I'm not sure how I feel about this.",
    "WOW!!! This is incredible! 😍🔥",
    "What a terrible decision. 😡",
    "The stock is looking good but I have some doubts...",
    "BUY BUY BUY!!! 🚀💰"
]

# Analyze sentiment for each text
for text in texts:
    sentiment_score = sia.polarity_scores(text)["compound"]
    print(f"📝 Text: {text} | 📊 Sentiment Score: {sentiment_score}")


📝 Text: This is amazing! | 📊 Sentiment Score: 0.6239
📝 Text: This is the worst experience ever. | 📊 Sentiment Score: -0.6249
📝 Text: I'm not sure how I feel about this. | 📊 Sentiment Score: -0.2411
📝 Text: WOW!!! This is incredible! 😍🔥 | 📊 Sentiment Score: 0.7718
📝 Text: What a terrible decision. 😡 | 📊 Sentiment Score: -0.4767
📝 Text: The stock is looking good but I have some doubts... | 📊 Sentiment Score: 0.2382
📝 Text: BUY BUY BUY!!! 🚀💰 | 📊 Sentiment Score: 0.0


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\nnodu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
#Using AG FROM HUGGINGFACE PLATFORM

# 📌  Load the AG News dataset from Hugging Face
from datasets import load_dataset

print("🔄 Loading AG News dataset...")
dataset = load_dataset("ag_news", split="train")
df = pd.DataFrame(dataset)
print(f"✅ Loaded {len(df)} news entries.")



# ✅ Step 4: Load the AG News dataset from Hugging Face
print("🔄 Loading AG News dataset...")
dataset = load_dataset("ag_news", split="train")
df = pd.DataFrame(dataset)
print(f"✅ Loaded {len(df)} news entries.")

# ✅ Step 5: Filter for Business (2) and Sci/Tech (3) news only
df = df[df["label"].isin([2, 3])]  # Business or Sci/Tech

# ✅ Step 6: Filter for Intel-related articles (case-insensitive)
intel_news = df[df["text"].str.contains("Intel", case=False, na=False)].copy()

# ✅ Step 7: Simulate recent dates (AG News doesn't include dates)
# We'll simulate with today's date minus some days
today = datetime.datetime.now()
intel_news["date"] = [today - datetime.timedelta(days=i) for i in range(len(intel_news))]

print(f"\n📌 Found {len(intel_news)} Intel-related news articles in AG News.\n")
print(intel_news[["date", "text"]].head())

# ✅ Step 8: Apply VADER sentiment analysis
results = []

for _, row in intel_news.iterrows():
    text = row["text"]
    sentiment = sia.polarity_scores(text)
    results.append({
        "date": row["date"].strftime("%Y-%m-%d"),
        "headline": row["text"],
        "compound": sentiment["compound"],
        "positive": sentiment["pos"],
        "neutral": sentiment["neu"],
        "negative": sentiment["neg"]
    })

# ✅ Step 9: Create sentiment results DataFrame
sentiment_df = pd.DataFrame(results)
sentiment_df = sentiment_df.sort_values(by="date", ascending=False)

# ✅ Step 10: Display sample results
print("\n🧾 Sample Sentiment Scores:\n")
print(sentiment_df.head())

# ✅ Step 11: Calculate average sentiment bias
ag_sentiment_avg = sentiment_df["compound"].mean()
print(f"\n🧠 Average VADER Sentiment from AG News: {ag_sentiment_avg:.4f}")


# ✅ Step 12: Interpret sentiment bias for long-term trading strategy
if ag_sentiment_avg > 0.1:
    bias = "📈 Generally Positive Bias (Bullish Long-Term Sentiment)"
elif ag_sentiment_avg < -0.1:
    bias = "📉 Generally Negative Bias (Bearish Long-Term Sentiment)"
else:
    bias = "🔄 Neutral or Mixed Bias (No clear sentiment direction)"

print(f"📊 Sentiment Bias Interpretation: {bias}")



🔄 Loading AG News dataset...
✅ Loaded 120000 news entries.
🔄 Loading AG News dataset...
✅ Loaded 120000 news entries.

📌 Found 1301 Intel-related news articles in AG News.

                          date  \
3   2025-03-29 17:18:04.867885   
70  2025-03-28 17:18:04.867885   
385 2025-03-27 17:18:04.867885   
553 2025-03-26 17:18:04.867885   
741 2025-03-25 17:18:04.867885   

                                                  text  
3    Iraq Halts Oil Exports from Main Southern Pipe...  
70   U.K.'s NHS taps Gartner to help plan \$9B IT o...  
385  Intel Shows Wireless Transceiver Prototype 90-...  
553  NASA Develops Robust Artificial Intelligence f...  
741  Computers with multiple personalities The jury...  

🧾 Sample Sentiment Scores:

         date                                           headline  compound  \
0  2025-03-29  Iraq Halts Oil Exports from Main Southern Pipe...   -0.1531   
1  2025-03-28  U.K.'s NHS taps Gartner to help plan \$9B IT o...    0.7003   
2  2025-03-27  In

In [7]:
# ✅ Your Finnhub API Key
FINNHUB_API_KEY = "cv6ibshr01qi7f6qh2cgcv6ibshr01qi7f6qh2d0"

#USING FINNHUB 
# ✅ Function to fetch Intel-related news from Finnhub (past 1 year)
def get_intel_news():
    symbol = "INTC"
    today = datetime.datetime.now().strftime('%Y-%m-%d')
    one_year_ago = (datetime.datetime.now() - datetime.timedelta(days=7)).strftime('%Y-%m-%d')

    url = f"https://finnhub.io/api/v1/company-news?symbol={symbol}&from={one_year_ago}&to={today}&token={FINNHUB_API_KEY}"
    response = requests.get(url).json()

    if not isinstance(response, list):
        print("⚠ Error: API response is not a list. Check API key or rate limit.")
        return []

    intel_news = []
    for article in response:
        intel_news.append({
            "headline": article.get("headline", ""),
            "summary": article.get("summary", ""),
            "datetime": datetime.datetime.fromtimestamp(article["datetime"]),
            "url": article.get("url", "")
        })

    return intel_news

# ✅ Step 1: Fetch Intel news
news_list = get_intel_news()

# ✅ Step 2: Display preview before sentiment analysis
if news_list:
    preview_df = pd.DataFrame(news_list).sort_values(by="datetime", ascending=False)
    
    print(f"\n🔎 Found {len(preview_df)} Intel-related news articles from Finnhub (past 1 year).\n")
    print("🗞️ Sample of Raw News Data:\n")
    print(preview_df[["datetime", "headline", "summary", "url"]].head(5))
else:
    print("⚠ No Intel-related news found to preview.")
    

# Apply sentiment analysis using VADER
def analyze_sentiment(news_list):
    sentiment_results = []
    for news in news_list:
        full_text = news["headline"] + " " + news.get("summary", "")
        sentiment_score = sia.polarity_scores(full_text)['compound']
        sentiment_results.append({
            "date": news["datetime"].strftime("%Y-%m-%d"),
            "headline": news["headline"],
            "summary": news["summary"],
            "sentiment_score": sentiment_score,
            "url": news["url"]
        })
    return sentiment_results

# Analyze the news
sentiment_data = analyze_sentiment(news_list)
sentiment_df = pd.DataFrame(sentiment_data).sort_values(by="date", ascending=False)

# Display sample results
print("\n📊 Sample Sentiment Analysis Results:\n")
print(sentiment_df[["date", "headline", "sentiment_score"]].head())


# Calculate average sentiment bias
finnhub_sentiment_avg = sentiment_df["sentiment_score"].mean()
print(f"\n🧠 Average VADER Sentiment from Finnhub: {finnhub_sentiment_avg:.4f}")


# Interpret bias
if finnhub_sentiment_avg > 0.1:
    bias = "📈 Bullish Bias (Positive Media Sentiment)"
elif finnhub_sentiment_avg < -0.1:
    bias = "📉 Bearish Bias (Negative Media Sentiment)"
else:
    bias = "🔄 Neutral Bias (Mixed Sentiment)"

print(f"📊 Bias Interpretation: {bias}")


🔎 Found 76 Intel-related news articles from Finnhub (past 1 year).

🗞️ Sample of Raw News Data:

             datetime                                           headline  \
0 2025-03-28 20:19:46  I'm Betting On Tan's Intel For A Trade In 2025...   
1 2025-03-28 12:25:49  Intel Exits. SK hynix Takes Over. A New Flash ...   
2 2025-03-28 12:19:25  Intel (NasdaqGS:INTC) Faces Shareholder Propos...   
3 2025-03-28 12:16:00  Intel Received $1.9 Billion From Final Close o...   
4 2025-03-28 11:11:49  Intel Board Members Leaving as Chipmaker Looks...   

                                             summary  \
0  Intel's new CEO, Lip-Bu Tan, is driving effici...   
1  The $8.85B deal that reshapes enterprise stora...   
2  Intel (NasdaqGS:INTC) experienced a 16% price ...   
3  Intel completed the second and final closing f...   
4  (Bloomberg) -- Intel Corp. said three members ...   

                                                 url  
0  https://finnhub.io/api/news?id=6aed07c24fbc2be... 

In [ ]:
import MetaTrader5 as mt5
#Connect to MT5
mt5.initialize()





# Trading Bot Configuration
ticker = 'INTC'
pip_size = 0.0001  # 1 pip for GBPUSD
qty = 0.16  # Lot Size

buy_order_type = mt5.ORDER_TYPE_BUY
sell_order_type = mt5.ORDER_TYPE_SELL

buy_price = mt5.symbol_info_tick("INTC").ask
sell_price = mt5.symbol_info_tick("INTC").bid

# Dictionaries
buy_signals = {'Buy_Signal': buy_price}
sell_signals = {'Sell_Signal': sell_price}

# Stop-Loss & Take-Profit for Buy Trades
buy_sl = buy_signals['Buy_Signal'] - (6 * pip_size)  # 6-pip SL
buy_tp = buy_signals['Buy_Signal'] + (12 * pip_size)  # 12-pip TP

# Stop-Loss & Take-Profit for Sell Trades
sell_sl = sell_signals['Sell_Signal'] + (6 * pip_size)  # 6-pip SL
sell_tp = sell_signals['Sell_Signal'] - (12 * pip_size)  # 12-pip TP


# Function to Create Either Buy or Sell Orders
def create_order(ticker, qty, order_type, price, sl, tp):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": qty,
        "type": order_type,
        "price": price,
        "sl": sl,
        "tp": tp,
        "comment": "Python open position",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }
    
    # Send the trading request
    order = mt5.order_send(request)
    
    if order and order.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"Order placed successfully: Ticket {order.order}")
        return order.order  # Return the trade ID
    else:
        print(f"Order failed: {order.comment if order else 'No response from server'}")
        return None


# Function for Closing Buy or Sell Orders
def close_order(ticker, qty, order_type, price):
    positions = mt5.positions_get(symbol=ticker)  # Get open positions for this ticker
    
    if positions:  # If there is an open trade
        position_id = positions[0]._asdict()['ticket']  # Get first open position's ticket
        
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": ticker,
            "volume": qty,
            "type": order_type,
            "price": price,
            "position": position_id,
            "comment": "Closing trade",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC
        }

        # Send the trading request
        order = mt5.order_send(request)
        
        if order and order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"Position {position_id} closed successfully")
        else:
            print(f"Failed to close position {position_id}: {order.comment if order else 'No response from server'}")
    else:
        print(f"No open positions to close for {ticker}")

        
        

        

        
        
        
import time

#TRADINGBOT
#Pine Script

while True:
    symbol = "INTC"
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M5, 0, 1000)  # Fetch more historical data
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    
        # Apply detection functions
    df = detect_support_resistance(df, window=10)
    df = detect_breakout_retest(df)
    df = detect_trade_signals(df)
    
    # Ensure 'Stop_Loss' & 'Take_Profit' exist before using them
    if 'Stop_Loss' not in df.columns:
        df['Stop_Loss'] = np.nan  
    if 'Take_Profit' not in df.columns:
        df['Take_Profit'] = np.nan 
    
    df['Stop_Loss'].fillna(0, inplace=True)
    df['Take_Profit'].fillna(0, inplace=True)


    long_condition = (
        pd.notna(df['Resistance'].shift(2)).any()  # Resistance exists 2 candles ago
        and (df['close'].shift(1) > df['Resistance'].shift(2)).any()  # Breakout confirmed
        and (df['low'] <= df['Resistance'].shift(2)).any()  # Retest confirmation
    )
    short_condition = (
        pd.notna(df['Support'].shift(2)).any()  # Support exists 2 candles ago
        and (df['close'].shift(1) < df['Support'].shift(2)).any()  # Breakout confirmed
        and (df['high'] >= df['Support'].shift(2)).any()  # Retest confirmation
    )
    close_long_condition = (
        (df['close'] <= df['Stop_Loss']).any() |  # Stop-loss reached
        (df['close'] >= df['Take_Profit']).any()  # Take-profit reached
    )
    close_short_condition = (
        (df['close'] >= df['Stop_Loss']).any() |  # Stop-loss reached for short
        (df['close'] <= df['Take_Profit']).any()  # Take-profit reached for short
    )
    
    # Ensure 'Breakeven_Level' exists before using it
    if 'Breakeven_Level' not in df.columns:
        df['Breakeven_Level'] = np.nan  # Initialize with NaN to prevent KeyErrors

    # Fill missing Breakeven Levels with 0
    df['Breakeven_Level'].fillna(0, inplace=True)
    
    breakeven_buy_condition = (
        df['high'] >= df['Breakeven_Level']
    ).any()
    breakeven_sell_condition = (
        df['low'] <= df['Breakeven_Level']
    ).any()
    
    already_buy = False
    already_sell = False
    
    try:
        already_sell = mt5.positions_get()[0]._asdict()['type'] == 1
        already_buy = mt5.positions_get()[0]._asdict()['type'] == 0
    except:
        pass
    
    no_positions = len(mt5.positions_get()) == 0
    
    # 🟢 RECORD TRADE OPEN TIME HERE
    trade_open_time = None

    #Implementing conditions for longs and shorts
    if long_condition:
        if no_positions:
            create_order(ticker, qty, buy_order_type, buy_price, buy_sl, buy_tp)
            print('Buy Order Placed')
            trade_open_time = time.time()  # Track when trade opens ✅
        if already_sell:
            close_order(ticker, qty, buy_order_type, buy_price)
            print('Sell Position Closed')
            time.sleep(1)
            create_order(ticker, qty, buy_order_type, buy_price, buy_sl, buy_tp)
            print('Buy Order Placed')
            trade_open_time = time.time()  # Track when trade opens ✅

    if short_condition:
        if no_positions:
            create_order(ticker, qty, sell_order_type, sell_price, sell_sl, sell_tp)
            print('Sell Order Placed')
            trade_open_time = time.time()  # Track when trade opens ✅
        if already_buy:
            close_order(ticker, qty, sell_order_type, sell_price)
            print('Buy Position Closed')
            time.sleep(1)
            create_order(ticker, qty, sell_order_type, sell_price, sell_sl, buy_tp)
            print('Sell Order Placed')
            trade_open_time = time.time()  # Track when trade opens ✅
    
    try:
        already_sell = mt5.positions_get()[0]._asdict()['type'] == 1
        already_buy = mt5.positions_get()[0]._asdict()['type'] == 0
    except:
        pass
    # 🟢 CHECK EXIT CONDITIONS ONLY AFTER HOLDING FOR 60 SECONDS ✅
    if trade_open_time and time.time() - trade_open_time > 60:
        if close_long_condition and already_buy:
            close_order(ticker, qty, sell_order_type, sell_price)
            print('Closed Buy Position')

        if close_short_condition and already_sell:
            close_order(ticker, qty, buy_order_type, buy_price)
            print('Closed Sell Position') 
    
    already_buy = False
    already_sell = False
    
    #Implementing condition for breakeven
    # Initialize hit_be before using it
    hit_be = pd.DataFrame()  # Ensures it's always defined

    # Implementing condition for breakeven
    if not hit_be.empty and (hit_sl.empty or hit_be.iloc[0]['time'] < hit_sl.iloc[0]['time']):
        if trade_type == 'Buy':
            buy_breakevens += 1
        else:
            sell_breakevens += 1
    
    time.sleep(10)
    
